In [1]:
import arcpy, os
import arcpy
from osgeo import gdal

arcpy.env.addOutputsToMap = False   # Prevents many small grid files to show on map
arcpy.env.overwriteOutput = True

In [2]:
# Directory option 1: Use project gdb
# Directory option 2: use project directory
# Directory option 3: Custom directory
Dir = 2  # [1, 2 or 3] (Default is 1)

if Dir == 1:
    p = arcpy.mp.ArcGISProject("CURRENT")
    directory = p.defaultGeodatabase   
elif Dir == 2:
    directory = os.path.dirname( arcpy.mp.ArcGISProject("CURRENT").filePath )
elif Dir == 3:
    directory = r"H:\DATA\Service\Wietse\GIS\DEM"

arcpy.env.workspace = directory

# Inladen bestanden

In [7]:
Input_Arcpy_Eenmalig = r"H:\Team_Kennis_OSA\Hydraulische toetsing\Basisbestanden Zomer\Input_ArpPy\Eenmalige_bestanden"
Bodemkaart_HDSR = os.path.join(Input_Arcpy_Eenmalig,"Bodemkaart_HDSR.shp")

In [9]:
import arcpy

# Set up your workspace and input data

input_line = "Primaire_watergangen.shp"
input_points = "Network_n_filter.shp"
output_lines = "output_lines.shp"

# Execute the Split Line at Point tool
arcpy.SplitLineAtPoint_management(input_line, input_points, output_lines)

print("Line split completed.")

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000824: The tool is not licensed.
Failed to execute (SplitLineAtPoint).


# Inladen Sobek bestanden

In [6]:
import arcpy

def split_line_with_point(line_shapefile, point_shapefile, output_shapefile):
    # Create an in-memory feature layer for the line shapefile
    arcpy.MakeFeatureLayer_management(line_shapefile, "line_layer")

    # Create a search cursor to iterate over the point features
    with arcpy.da.SearchCursor(point_shapefile, ["SHAPE@"]) as cursor:
        for row in cursor:
            point_geometry = row[0]

            # Select the line features intersecting the current point
            arcpy.SelectLayerByLocation_management("line_layer", "INTERSECT", point_geometry)

            # Create an empty list to store the split line features
            split_lines = []

            # Iterate over the selected line features
            with arcpy.da.SearchCursor("line_layer", ["SHAPE@"]) as line_cursor:
                for line_row in line_cursor:
                    line_geometry = line_row[0]

                    # Split the line at the point using geometry operations
                    left_part = line_geometry.segmentAlongLine(0, point_geometry)
                    right_part = line_geometry.segmentAlongLine(point_geometry, line_geometry.length)

                    # Add the split line features to the list
                    split_lines.append(left_part)
                    split_lines.append(right_part)

            # Insert the split line features into the output shapefile
            arcpy.CopyFeatures_management(split_lines, output_shapefile)

    # Clean up the in-memory feature layer
    arcpy.Delete_management("line_layer")

# Example usage
line_shapefile = os.path.join(directory,"RchSegments.shp")
point_shapefile = os.path.join(directory,"Network_n_filter.shp")
output_shapefile = os.path.join(directory,"test.shp")

split_line_with_point(line_shapefile, point_shapefile, output_shapefile)


RuntimeError: Object: Error in executing tool

In [12]:
def generate_points_at_segment_end(input_line_shapefile, output_point_shapefile):
    # Set environment settings
    arcpy.env.workspace = directory

    # Create a new point shapefile
    arcpy.CreateFeatureclass_management(arcpy.env.workspace, "temp_points.shp", "POINT", input_line_shapefile)

    # Iterate over line features
    with arcpy.da.SearchCursor(input_line_shapefile, ["SHAPE@", "OID@"]) as cursor:
        for row in cursor:
            line = row[0]
            line_oid = row[1]

            # Extract start and end points
            start_point = line.firstPoint
            end_point = line.lastPoint

            # Create new point feature and insert it into the temporary point shapefile
            point = arcpy.PointGeometry(end_point)
            with arcpy.da.InsertCursor("temp_points.shp", "SHAPE@") as icursor:
                icursor.insertRow([point])

    # Delete all line features from the input line shapefile
    arcpy.DeleteFeatures_management(input_line_shapefile)

    # Merge the temporary point shapefile with the output point shapefile
    arcpy.Merge_management(["temp_points.shp", output_point_shapefile], "in_memory/merged_points")

    # Rename the merged point shapefile to the desired output name
    arcpy.Rename_management("in_memory/merged_points", output_point_shapefile)

    # Delete the temporary point shapefile
    arcpy.Delete_management("temp_points.shp")



In [13]:
input_line_shapefile = "RchSegments.shp"
output_point_shapefile = "Points_sobek.shp"
generate_points_at_segment_end(input_line_shapefile, output_point_shapefile)


ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input Datasets: Dataset Points_sobek.shp does not exist or is not supported
Failed to execute (Merge).


# Bepalen boden voor stroomsnelheid

In [9]:
arcpy.AddField_management(Bodemkaart_HDSR, "vel_crit", "DOUBLE")

def calculate_value(field_value):
    if "Stedelijk" in field_value or "stedelijk" in field_value:
        return 0.3
    elif "Zand" in field_value or "zand" in field_value:
        return 0.3
    elif "Veen" in field_value or "veen" in field_value:
        return 0.3001
    elif "Klei" in field_value or "klei" in field_value:
        return 0.6
    elif "Zavel" in field_value or "zavel" in field_value:
        return 0.6
    elif "Water" in field_value or "water" in field_value:
        return 0.3
    elif "Moeras" in field_value or "moeras" in field_value:
        return 0.3001
    elif "Overig" in field_value or "overig" in field_value:
        return 0.3001
    else:
        return 0.9

# Calculate the values of the calculation_field using the calculate_value function
with arcpy.da.UpdateCursor(Bodemkaart_HDSR, ["GRONDSOORT", "vel_crit"]) as cursor:
    for row in cursor:
        row[1] = calculate_value(row[0])
        cursor.updateRow(row)


In [ ]:
def perform_spatial_join(input_features, join_features, output_features):
    field_mappings = arcpy.FieldMappings()
    field_mappings.addTable(input_features)
    field_mappings.addTable(join_features)

    output_fields = ["ID", "hydro_segm", "pg_code", "Value", "dir_et", "LENGTE", "vel_crit"]

    for field in field_mappings.fields:
        if field.name not in output_fields:
            field_mappings.removeFieldMap(field_mappings.findFieldMapIndex(field.name))

    arcpy.SpatialJoin_analysis(
        target_features=input_features,
        join_features=join_features,
        out_feature_class=output_features,
        join_operation="JOIN_ONE_TO_ONE",
        join_type="KEEP_ALL",
        field_mapping=field_mappings,
        match_option="INTERSECT",
        search_radius="0 Meters"
    )

# Example usage:
input_features = "SOBEK_ReachSegments_Join_PG_Hydro_Line_20210819_Discharge_Length"
output_features = "SOBEK_ReachSegments_Join_PG_Hydro_Line_20210819_Discharge_Length_Soil_shp"

perform_spatial_join(input_features, Bodemkaart_HDSR, output_features)


# Drooglegging bepalen

# Droogleggingseis toetsen per punt

# Exporteer Sobek resultaten naar kunstwerk punten